In [34]:
import numpy as np
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [35]:
from mcda.matrices import PerformanceTable
from mcda.scales import QuantitativeScale, PreferenceDirection
from mcda.outranking.electre import Electre3
from mcda.outranking.promethee import Promethee1, Promethee2
import numpy as np
from mcda.functions import VShapeFunction
from mcda.relations import PreferenceStructure
from helper_functions import dict_to_matrix, dict_to_matrix_with_prefix, net_flow_score, net_flow_score_pos_neg, create_preference_matrix, outranking_ranking, resolve_matrix_preferences, visualize_outranking

from new_methods.crisp_partial import crisp_partial
from new_methods.electre_partial import electre_partial
from new_methods.promethee_partial import promethee_partial

from new_methods.crisp_complete import crisp_complete
from new_methods.electre_complete import electre_complete
from new_methods.promethee_complete import promethee_complete

In [36]:
def calculate_incomparabilities(ranking_matrix):
    # Calculate incomparabilities (0 to 0) excluding diagonal elements
    incomparabilities = 0
    n = ranking_matrix.shape[0]
    for i in range(n):
        for j in range(i + 1, n):  # Only upper triangle to avoid double counting
            if ranking_matrix[i, j] == 0 and ranking_matrix[j, i] == 0:
                incomparabilities += 1
    return incomparabilities

In [37]:
def calculate_indifferences(ranking_matrix):
    # Calculate indifferences (1 to 1) excluding diagonal elements
    indifferences = 0
    n = ranking_matrix.shape[0]
    for i in range(n):
        for j in range(i + 1, n):  # Only upper triangle to avoid double counting
            if ranking_matrix[i, j] == 1 and ranking_matrix[j, i] == 1:
                indifferences += 1
    return indifferences

In [38]:
def generate_grid_parameters():
    samples = []
    for price in range(1000, 4200, 200):
        for memory_size in [64, 128, 256, 512]:
            for camera in [16, 32, 48, 64, 120]:
                for functionalities in range(2, 5, 1):
                    for battery_life in range(12, 32, 2):
                        samples.append([price, memory_size, camera, functionalities, battery_life])
    return samples


samples = generate_grid_parameters()
len(samples) 

9600

In [39]:
# def generate_grid_weights():
#     samples = []
#     for price in range(1, 20, 1):
#         for memory_size in range(1, 20, 1):
#             for camera in range(1, 20, 1):
#                 for functionalities in range(1, 20, 1):
#                     for battery_life in range(1, 10, 1):
#                         if price != memory_size != camera != functionalities != battery_life:
#                             samples.append([price, memory_size, camera, functionalities, battery_life])
#     return samples

# # This function will generate a lot of combinations, so be mindful of the ranges and increments.
# samples = generate_grid_weights()
# len(samples) 

In [40]:
def get_example(sample):


    alternatives = [
        "A1",
        "A2",
        "A3",
        "A4",
        "A5",
        "A6",
        "A7"
    ]

    # Definicja skali dla każdego kryterium
    scales = {
        0: QuantitativeScale(1000, 4500, preference_direction=PreferenceDirection.MIN),# Price 
        1: QuantitativeScale(32, 1024, preference_direction=PreferenceDirection.MAX),    # Memory Size
        2: QuantitativeScale(8, 120, preference_direction=PreferenceDirection.MAX),     # Camera 
        3: QuantitativeScale(0, 5, preference_direction=PreferenceDirection.MAX),      # Functionalities 
        4: QuantitativeScale(12, 32, preference_direction=PreferenceDirection.MAX),      # Battery life
    }

    cutoff = 0.7 # threshold

    dataset = PerformanceTable(
        [sample[0],
        sample[1],
        sample[2],
        sample[3],
        sample[4],
        sample[5],
        sample[6]],
        alternatives=alternatives, scales=scales
    )
    # Wagi kryteriów
    W = {
        0: 8,  # Price 
        1: 4,   # Memory Size
        2: 7,  # Camera 
        3: 2,  # Functionalities 
        4: 5   # Battery life
    }   
    
    I = {0: 300, 1: 64,  2: 12, 3: 2,      4: 2}
    P = {0: 600, 1: 256, 2: 32, 3: 3,      4: 6}
    V = {0: 900, 1: 512, 2: 64, 3: np.inf, 4: 12}

    preference_func_list = {0: VShapeFunction(p=600, q=300),
                        1: VShapeFunction(p=128, q=64),
                        2: VShapeFunction(p=32, q=12),
                        3: VShapeFunction(p=3, q=2),
                        4: VShapeFunction(p=6, q=2)}
    
    promethee1 = Promethee1(dataset, W, preference_func_list)

    promethee_matrix = promethee1.preferences(promethee1.partial_preferences()).data; promethee_matrix

    # Parametry Electre III

    electre3 = Electre3(performance_table=dataset, 
                    criteria_weights=W, 
                    indifference_thresholds=I, 
                    preference_thresholds=P, 
                    veto_thresholds=V)
    electre_matrix = electre3.construct().data
    electre_binary_matrix = (electre_matrix >= cutoff).astype(int)

    electre_crisp_complete = crisp_complete(electre_binary_matrix.values)
    electre_crisp_partial = crisp_partial(electre_binary_matrix.values)

    new_electre_partial = electre_partial(electre_matrix.values)
    new_electre_complete = electre_complete(electre_matrix.values)

    new_promethee_partial = promethee_partial(promethee_matrix.values)
    new_promethee_complete = promethee_complete(promethee_matrix.values)


    return [dict_to_matrix(electre_crisp_complete['solution_r']).astype(int), electre_crisp_complete['objective_value'], 
            dict_to_matrix(electre_crisp_partial['solution_r']).astype(int), electre_crisp_partial['objective_value'], 
            dict_to_matrix(new_electre_partial['solution_r']).astype(int), new_electre_partial['objective_value'], 
            dict_to_matrix(new_electre_complete['solution_r']).astype(int), new_electre_complete['objective_value'], 
            dict_to_matrix(new_promethee_partial['solution_r']).astype(int), new_promethee_partial['objective_value'], 
            dict_to_matrix(new_promethee_complete['solution_r']).astype(int), new_promethee_complete['objective_value']]
    #new_electre_binary_complete = crisp_complete(electre_binary_matrix.values)

In [41]:
random.shuffle(samples) 

In [42]:
samples[:7]

[[2600, 512, 48, 4, 12],
 [1600, 512, 120, 3, 16],
 [3000, 128, 48, 3, 18],
 [2200, 256, 32, 4, 22],
 [3400, 512, 48, 3, 14],
 [2800, 512, 64, 2, 14],
 [3600, 64, 120, 4, 24]]

In [43]:
def test_combinations_until_found(samples):
    for i in  range(100000):
        if i%1000 == 0:
            print(i)

        random.shuffle(samples) 
        sample = samples[:7]

        results = get_example(sample)
        c_complete, obj_1, c_partial, obj_2, e_complete, obj_3, e_partial, obj_4, p_complete, obj_5, p_partial, obj_6 = results[0],results[1],results[2],results[3],results[4],results[5],results[6],results[7],results[8],results[9],results[10],results[11]
        # Simulate ranking generation for the 6 selected samples

        
        # Calculate incomparabilities and indifferences
        cpr = calculate_incomparabilities(c_partial)
        cpi = calculate_indifferences(c_partial)

        epr = calculate_incomparabilities(e_partial)
        epi = calculate_indifferences(e_partial)

        ppr = calculate_incomparabilities(p_partial)
        ppi = calculate_indifferences(p_partial)

        ccr = calculate_indifferences(c_complete)
        ecr = calculate_indifferences(e_complete)
        pcr = calculate_indifferences(p_complete)



        if (obj_1 > 6 and obj_2 > 6 and obj_3 > 6 and obj_4 > 6 and obj_5 > 6 and obj_6 > 6) and \
            cpr >= 1 and cpr <= 7 and \
            cpi >= 1 and cpi <= 7 and \
            ppr >= 1 and \
            ppi >= 1 and ppi <= 6:
           
            # and \
        
            # epr >= 1 and epr <= 6 and \
            # epi >= 1 and epi <= 5 and \
            
            # ccr >= 1 and ccr <= 5 and \
            # ecr >= 1 and ecr <= 5 and \
            # pcr >= 1 and pcr <= 5:
            print('<==========>')
            print(sample)
            # print('complete')
            # print(ranking_matrix_complete)
            print('<==========>')
        

In [ ]:
test_combinations_until_found(samples)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
